In [1]:
!pip install pymorphy2[fast]
!pip install pymystem3
!pip install rusenttokenize
!pip install razdel

qlearning 1.0.0 requires nesgym-super-mario-bros>=0.3.1, which is not installed.
You are using pip version 10.0.1, however version 20.3b1 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.


qlearning 1.0.0 requires nesgym-super-mario-bros>=0.3.1, which is not installed.
You are using pip version 10.0.1, however version 20.3b1 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.


In [2]:
from pprint import pprint

In [3]:
import re
from pymorphy2 import MorphAnalyzer
from pymystem3 import Mystem
from nltk.corpus import stopwords
from razdel import tokenize as razdel_tokenize
from rusenttokenize import ru_sent_tokenize
from string import punctuation
from nltk.stem.snowball import SnowballStemmer
from collections import Counter

In [5]:
with open("zhivago.txt", "r", encoding="UTF-8") as file:
    chunk: str = file.read()
    chunk = chunk.lower()

## Удаляем мусор. Не удаляем знаки препинания во избежание ошибок при делении на предложения.

In [11]:
def clear_func(string):
    newstring = string[:]
    newstring = re.sub(r'<[^>]+>', ' ', newstring)
    newstring = re.sub(r'(\n|\xa0)', ' ', newstring)
    newstring = re.sub(r' {2,}', ' ', newstring)
    newstring = re.sub(r'[a-zA-Z]+', '', newstring)
    return newstring

In [12]:
cleaned = clear_func(chunk)

## Отрезаем лишние символы в конце файла

In [13]:
cleaned = cleaned[:cleaned.find("/9j/")]

In [14]:
cleaned[:2000]

' борис леонидович пастернак доктор живаго «доктор живаго» - итоговое произведение бориса пастернака, книга всей его жизни. этот роман принес его автору мировую известность и нобелевскую премию, присуждение которой обернулось для поэта оголтелой политической травлей, обвинениями в «измене родине» и в результате стоило ему жизни. «доктор живаго» - роман, сама ткань которого убедительнее свидетельствует о чуде, чем все размышления доктора и обобщения автора. человек, который так пишет, бесконечно много пережил и передумал, и главные его чувства на свете - восхищенное умиление и слезное сострадание; конечно, есть в его мире место и презрению, и холодному отстранению - но не в них суть. роман пастернака - оплакивание прежних заблуждений и их жертв; те, кто не разделяет молитвенного восторга перед миром, достойны прежде всего жалости. перечитывать «доктора живаго» стоит именно тогда, когда кажется, что жить не стоит. тогда десять строк из этого романа могут сделать то же, что делает любовь 

## Деление на предложения

In [15]:
sentences = ru_sent_tokenize(cleaned)

In [17]:
print("\n\n".join(sentences[:5]))

борис леонидович пастернак доктор живаго «доктор живаго» - итоговое произведение бориса пастернака, книга всей его жизни.

этот роман принес его автору мировую известность и нобелевскую премию, присуждение которой обернулось для поэта оголтелой политической травлей, обвинениями в «измене родине» и в результате стоило ему жизни.

«доктор живаго» - роман, сама ткань которого убедительнее свидетельствует о чуде, чем все размышления доктора и обобщения автора.

человек, который так пишет, бесконечно много пережил и передумал, и главные его чувства на свете - восхищенное умиление и слезное сострадание; конечно, есть в его мире место и презрению, и холодному отстранению - но не в них суть.

роман пастернака - оплакивание прежних заблуждений и их жертв; те, кто не разделяет молитвенного восторга перед миром, достойны прежде всего жалости.


In [16]:
def clean_punct(string: str) -> str:
    """Удаление всех знаков, кроме дефиса + дефисов не в середине слова
    :param string: Строка со знаками препинания
    :return: Очищенная строка.
    """
    string = string.lower()
    string = re.sub(r'[^а-я А-Я0-9-]', "", string)
    string = re.sub(r'(?<![а-яА-Я])-(?![а-яА-Я])', "", string)
    string = re.sub(r' {2,}', " ", string)
    string = re.sub(r'\d+', '', string) # Удаление цифр за ненадобостью
    return string

In [17]:
cleaned_sentences = [n for n in map(clean_punct, sentences)]

In [20]:
print("\n\n".join(cleaned_sentences[:5]))

борис леонидович пастернак доктор живаго доктор живаго итоговое произведение бориса пастернака книга всей его жизни

этот роман принес его автору мировую известность и нобелевскую премию присуждение которой обернулось для поэта оголтелой политической травлей обвинениями в измене родине и в результате стоило ему жизни

доктор живаго роман сама ткань которого убедительнее свидетельствует о чуде чем все размышления доктора и обобщения автора

человек который так пишет бесконечно много пережил и передумал и главные его чувства на свете восхищенное умиление и слезное сострадание конечно есть в его мире место и презрению и холодному отстранению но не в них суть

роман пастернака оплакивание прежних заблуждений и их жертв те кто не разделяет молитвенного восторга перед миром достойны прежде всего жалости


In [21]:
count = Counter(cleaned_sentences)

## В тексте много раз повторяется рефрен из стихотворения "Зимняя ночь". + Предложение "Толпа росла".

In [22]:
count.most_common(10)

[('свеча горела на столе свеча горела', 3),
 ('парило', 2),
 ('странно', 2),
 ('толпа росла', 2),
 ('да', 2),
 ('он открыл глаза', 2),
 ('борис леонидович пастернак доктор живаго доктор живаго итоговое произведение бориса пастернака книга всей его жизни',
  1),
 ('этот роман принес его автору мировую известность и нобелевскую премию присуждение которой обернулось для поэта оголтелой политической травлей обвинениями в измене родине и в результате стоило ему жизни',
  1),
 ('доктор живаго роман сама ткань которого убедительнее свидетельствует о чуде чем все размышления доктора и обобщения автора',
  1),
 ('человек который так пишет бесконечно много пережил и передумал и главные его чувства на свете восхищенное умиление и слезное сострадание конечно есть в его мире место и презрению и холодному отстранению но не в них суть',
  1)]

## Токенизация

In [18]:
alltokens = []
for i in cleaned_sentences:
    alltokens += [n.text for n in razdel_tokenize(i)]
alltokens[:30]

['борис',
 'леонидович',
 'пастернак',
 'доктор',
 'живаго',
 'доктор',
 'живаго',
 'итоговое',
 'произведение',
 'бориса',
 'пастернака',
 'книга',
 'всей',
 'его',
 'жизни',
 'этот',
 'роман',
 'принес',
 'его',
 'автору',
 'мировую',
 'известность',
 'и',
 'нобелевскую',
 'премию',
 'присуждение',
 'которой',
 'обернулось',
 'для',
 'поэта']

## Частотный токен длиннее 6 символов - "Андреевич", отчество героя.

In [24]:
all_count = Counter(alltokens)
all_count.most_common(50)

[('и', 6475),
 ('в', 4748),
 ('не', 2933),
 ('на', 2663),
 ('с', 1929),
 ('он', 1577),
 ('что', 1489),
 ('как', 1288),
 ('я', 1189),
 ('его', 1085),
 ('к', 1037),
 ('это', 1001),
 ('а', 990),
 ('по', 964),
 ('из', 796),
 ('все', 761),
 ('у', 729),
 ('но', 723),
 ('она', 719),
 ('за', 718),
 ('было', 612),
 ('ее', 602),
 ('от', 591),
 ('их', 572),
 ('о', 491),
 ('так', 468),
 ('они', 444),
 ('бы', 440),
 ('ты', 431),
 ('был', 426),
 ('вы', 381),
 ('же', 376),
 ('или', 366),
 ('когда', 364),
 ('еще', 352),
 ('до', 344),
 ('чтобы', 324),
 ('ему', 317),
 ('мне', 302),
 ('только', 293),
 ('под', 292),
 ('была', 292),
 ('то', 291),
 ('юрий', 289),
 ('андреевич', 289),
 ('были', 284),
 ('меня', 278),
 ('без', 262),
 ('вот', 262),
 ('да', 258)]

## Стемминг

In [25]:
stemmer = SnowballStemmer('russian')
stemmed_tokens = [n for n in map(stemmer.stem, alltokens)]
stemmed_tokens[:35]

['борис',
 'леонидович',
 'пастернак',
 'доктор',
 'живаг',
 'доктор',
 'живаг',
 'итогов',
 'произведен',
 'борис',
 'пастернак',
 'книг',
 'все',
 'ег',
 'жизн',
 'этот',
 'рома',
 'принес',
 'ег',
 'автор',
 'миров',
 'известн',
 'и',
 'нобелевск',
 'прем',
 'присужден',
 'котор',
 'обернул',
 'для',
 'поэт',
 'оголтел',
 'политическ',
 'травл',
 'обвинен',
 'в']

## Выделяем частотные стемы

In [43]:
count_stems = Counter(stemmed_tokens)
popular = count_stems.most_common(150)[50:]

In [ ]:
popular = [item for item in map(lambda x: x[0], popular)]


## Примеры стемов, совпавших с исходными токенами

In [52]:
popular2 = filter(lambda x: len(x) > 4, popular)
for stem in popular2:
    for index, word in enumerate(stemmed_tokens):
        if word == stem and alltokens[index] == stem:
            print(f"{stem} == {alltokens[index]}")
            break
# lookup = set(alltokens)
# no_change = [n for n in filter(lambda x: len(x) > 4 and x in lookup, stemmed_tokens[:1000])]
# no_change

дорог == дорог
сторон == сторон
комнат == комнат
перед == перед
человек == человек
может == может
город == город
будет == будет
хорош == хорош
через == через
минут == минут
поезд == поезд
сейчас == сейчас


## Различные окены с совпавшим стеммингом

In [48]:
equivalents = dict()
for n in popular[:20]:
    equivalents[n] = set()
    for index, item in enumerate(stemmed_tokens):
        if item == n:
            equivalents[n].add(alltokens[index])
pprint(equivalents.items())

dict_items([('под', {'поду', 'под', 'подо', 'поди', 'подите'}), ('теб', {'тебе', 'тебя'}), ('мен', {'мене', 'мена', 'менее', 'меня'}), ('тольк', {'только'}), ('рук', {'рука', 'рукам', 'рукой', 'руках', 'руки', 'руками', 'рук', 'рукою', 'руку', 'руке'}), ('жизн', {'жизнью', 'жизнь', 'жизней', 'жизни'}), ('больш', {'большом', 'большой', 'большим', 'большую', 'большая', 'большие', 'больший', 'большою', 'большей', 'больше', 'большею', 'большем', 'больших', 'большего', 'большими', 'большее', 'большого', 'большое', 'больша'}), ('да', {'дай', 'даем', 'даю', 'да'}), ('врем', {'времена', 'время', 'времен'}), ('без', {'без'}), ('вот', {'вот'}), ('лар', {'лар', 'лару', 'лары', 'ларою', 'лара', 'ларой', 'ларя', 'ларях', 'лари', 'ларе'}), ('мы', {'мы'}), ('тут', {'тут'}), ('для', {'для'}), ('тепер', {'теперь'}), ('сказа', {'сказались', 'сказать', 'сказали', 'сказанного', 'сказался', 'сказала', 'сказанные', 'сказал', 'сказанными', 'сказанное', 'сказаться', 'сказав', 'сказано'}), ('мо', {'моем', 'мое

## Примеры: Предлог под и глагол подите, слово мена и местоимение меня, слова ларь и Лара.

In [49]:
equivalents["под"]

{'под', 'поди', 'подите', 'подо', 'поду'}

In [50]:
equivalents["мен"]

{'мена', 'мене', 'менее', 'меня'}

In [51]:
equivalents["лар"]

{'лар',
 'лара',
 'ларе',
 'лари',
 'ларой',
 'ларою',
 'лару',
 'лары',
 'ларя',
 'ларях'}

In [60]:
import nltk

In [61]:
nltk.download("stopwords")

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [62]:
from nltk.corpus import stopwords

In [ ]:
stops = stopwords.words('russian')
pprint(stops)

### Слова для добавления
- как-то / как-либо (в списке нлтк нет стоп-слов через дефис, поэтому их можно все вносить в список).
- словно (употребляется преимущественно в сравнительных оборотах, поэтому может быть удалено наряду с "как").
- едва (употребляется в значении меры в любых контекстах, поэтому бесполезно для моделей на bag-of-words).
- однако (удаляется наряду с но, т.к. употребляется в сходных случаях).
- естественно (используется преимущественно как вводное слово, наряду с "конечно", имеющимся в корпусе).

## Токенизация

In [19]:
cash = {}
morphy_lemmatized = []
morph = MorphAnalyzer()
for word in alltokens:
    n = cash.get(word, False)
    if n:
        morphy_lemmatized.append(n)
    else:
        new = morph.parse(word)[0].normal_form
        cash[word] = new
        morphy_lemmatized.append(new)

In [27]:
cash = {}
mystem_lemmatized = []
mystem = Mystem(grammar_info=False, disambiguation=False)
for word in alltokens:
    n = cash.get(word, False)
    if n:
        mystem_lemmatized.append(n)
    else:
        new = mystem.lemmatize(word)
        cash[word] = new
        mystem_lemmatized.append(new)

In [29]:
mystem_lemmatized2 = [n for n in map(lambda x: x[0], mystem_lemmatized)]

In [30]:
print(morphy_lemmatized[:20])
print(mystem_lemmatized2[:20])

['борис', 'леонид', 'пастернак', 'доктор', 'живаго', 'доктор', 'живаго', 'итоговый', 'произведение', 'борис', 'пастернак', 'книга', 'весь', 'он', 'жизнь', 'этот', 'роман', 'принести', 'он', 'автор']
['борис', 'леонидович', 'пастернак', 'доктор', 'живаго', 'доктор', 'живаго', 'итоговый', 'произведение', 'борис', 'пастернак', 'книга', 'весь', 'он', 'жизнь', 'этот', 'роман', 'приносить', 'он', 'автор']


In [31]:
morphy_set = set(morphy_lemmatized)
mystem_set = set(mystem_lemmatized2)

## Для выявления разницы лемматизаторов рассмотрим разность множеств лемм

In [35]:
morphy_diff = morphy_set.difference(mystem_set)

In [36]:
mystem_diff = mystem_set.difference(morphy_set)

## Больше преимуществ у mystem:
### - pymorphy не учитывает pluralia tantum:
### поборы лемматизируются как "побор", этого нет в mystem

### - pymorphy хуже распознаёт номинализованные прилагательные: см. формы "чайная", "мертвецкая" которые не нашёл pymorphy

### - pymorphy хуже распознаёт отчества и имена собственные, которых нет в словаре: см. пример с "нефедыча", которое mystem распознает как "нефедыч".

### - mystem тяготеет к приведению форм совершенного вида к инфинитиву несовершенного вида: ср. парсинг "принес" в первом предложении как "принести" и "приносить" у pymorphy и mystem соответственно. Это приводит к подбору сомнительных форм (см. "нащеплять" от "нащепил").

In [38]:
"побор" in morphy_diff

True

In [39]:
"чайная" in mystem_diff

True

In [50]:
"мертвецкая" in mystem_diff

True

In [47]:
"нефедыча" in morphy_diff

True

In [49]:
"нефедыч" in mystem_diff

True

In [48]:
"жюля" in morphy_diff

True